In [38]:
%pip -q install google-genai

In [37]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_2')

In [39]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [40]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [125]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [42]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [43]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):

  if text is None:
    return Markdown("No content to display.") # Or handle as appropriate
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [148]:
##########################################
# --- Agente 1: Buscador de Referências --- #
##########################################
def agente_buscador(topico, data_de_hoje):

  buscador= Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      instruction="""
      Você é um analista de artes, com um grande conhecimento sobre diversas mídias, como arquitetura, escultura, pintura, música,
      literatura, dança, cinema, televisão, HQs, entre outros. A sua tarefa é usar a ferramenta de busca do Google (google_search) para identificar
      com base no nome de obras enviadas no tópico, as diferentes influências dos movimentos artísticos e obras que serviram de referência para a obra mencionada.
      """,
      description="Agente que busca informações no Google",
      tools=[google_search]

  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  referencias = call_agent(buscador, entrada_do_agente_buscador)
  return referencias

In [151]:
################################################
# --- Agente 2: Explicador --- #
################################################
def agente_explicador(topico, referencias_buscadas):
    explicador = Agent(
        name="agente_explicador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Explicador #################################################
        instruction="""
        Você é um historiador de todas as artes. Com base na lista de obras e movimentos relevantes buscados pelo agente buscador, você deve:
        usar a ferramenta de buscas do Google (google_search) para criar uma explicação sucinta, mas completa sobre cada uma delas, foque principalmente
        na parte dos movimentos artísticos, se possível evite obras do mesmo artista.
        Caso não seja possível, busque mais movimentos artísticos que influenciaram.
        """,
        description="Agente que explica as referências",
        tools=[google_search]
    )

    entrada_do_agente_explicador = f"Tópico:{topico}\nReferências buscadas: {referencias_buscadas}"
    # Executa o agente
    explicacao = call_agent(explicador, entrada_do_agente_explicador)
    return explicacao

In [138]:
######################################
# --- Agente 3: Indicador --- #
######################################
def agente_indicador(topico, indicação):
    indicador = Agent(
        name="agente_indicador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um grande acadêmico das artes, com um incrível repertório.
            Seu trabalho é indicar até 5 das mais relevantes obras pertencentes aos movimentos mencionados
            anteriormente.
            """,
        description="Agente indicador de obras que podem agregar no conhecimento"
    )
    entrada_do_agente_indicador = f"Tópico: {topico}\nPlano de post: {indicação}"
    # Executa o agente
    rascunho = call_agent(indicador, entrada_do_agente_indicador)
    return rascunho

In [ ]:
from datetime import date
# from IPython.display import display, Markdown # Supondo que você esteja usando Jupyter/Colab
# from your_module import agente_buscador # Supondo que agente_buscador esteja em um módulo chamado your_module
# from your_module import chat_2 # Supondo que chat_2 esteja em um módulo chamado your_module

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o sistema de busca de referências artísticas")

while True:
    # --- Obter o Tópico do Usuário ---
    topico = input("❓ Por favor, digite o nome da obra que deseja analisar (ou digite 'fim' para sair): ")

    if topico.lower() == "fim":
        print("Encerrando o sistema. Até a próxima!")
        break

    # Inserir lógica do sistema de agentes ################################################
    if not topico:
        print("Você esqueceu de digitar o tópico!")
    else:
        print(f"Maravilha! Vamos então buscar as referências de {topico}")

        referencias = agente_buscador(topico, data_de_hoje)
        print("\n--- Resultado do Agente 1 (Buscador) ---\n")
        display(to_markdown(referencias))
        print("----------------------------------------------------------------")

        indicação = agente_explicador(topico, referencias)
        print("\n--- Resultado do Agente 2 (explicador) ---\n")
        display(to_markdown(indicação))
        print("----------------------------------------------------------------")

        rascunho_de_post = agente_indicador(topico, indicação)
        print("\n--- Resultado do Agente 3 (indicador) ---\n")
        display(to_markdown(rascunho_de_post))
        print("----------------------------------------------------------------")

print("Fim da execução.")